- 실습중인 쇼핑몰 상품은 여러페이지에 걸쳐 진열되어 있음
    - 함수로 구성 후 여러페이지 크롤링

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
# context = ssl._create_unverified_context()
url="http://jolse.com/category/toners-mists/1019/"


### 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환값 --> bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달받아서
    - get_product_info(box)를 호출(제품 1개에 대한 정보가 반환)
    - df을 생성 후 저장
- 각 제품의 정보 추출 : get_product_info(box)
    - 제품정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [26]:
prd_dict={'품목': [],
         '가격':[],
         '세일가격': []}
df_fin = pd.DataFrame(prd_dict)

In [27]:
# 요청 및 파싱 객체 생성
def get_request_product(url):
    url = 'http://jolse.com/category/toners-mists/1019/'
    htmls = urlopen(url)
    htmls = htmls.read()
    # 2. 파싱객체 생성
    bs_obj = BeautifulSoup(htmls,'html.parser')
    
    return bs_obj

In [28]:
def get_page_product(url):
    global df_fin
    bs_obj = get_request_product(url)
    # 페이지내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    # 각 제품에 대한 정보 추출
    for box in boxes:
        res = pd.DataFrame(get_product_info(box),index=range(1,2))
        
        df_fin = pd.concat([df_fin,res],axis=0,ignore_index=True)

In [17]:
def get_product_info(box):
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    #가격
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text
    return {'품목':prd_name, '가격':price,'세일가격':sale_price}

In [18]:
url

'http://jolse.com/category/toners-mists/1019/'

In [7]:
get_page_product(url)

HTTPError: HTTP Error 502: Bad Gateway

In [ ]:
df_fin

##  여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착


In [ ]:
base_url ='https://jolse.com/category/toners-mists/1019?page= '
for i in range(2,5):
    url = base_url + str(i)
    get_page_product(url)

In [ ]:
df_fin

In [ ]:
# 수집 데이터 저장
df_fin.to_csv('./crawl_data/beauty_price.csv')

## 연습문제
- best 상품에 대해서 첫번째 페이지에서만 추출되고 그 후 페이지에서는 추출되지 않도록 함수 수정
- 전체 페이지가 몇페이지인지를 크롤링을 통해서 얻어오고 해당 페이지 수 만큼 크롤링이 되게 코드 수정


In [47]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
# context = ssl._create_unverified_context()


In [48]:
# 요청 및 파싱 객체 생성
def get_request_product(url):
   
    htmls = urlopen(url)
    htmls = htmls.read()
    # 2. 파싱객체 생성
    bs_obj = BeautifulSoup(htmls,'html.parser')
    
    return bs_obj

In [49]:
# 한 개의 제품에 대한 정보 추출
def get_product_info(box):
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    #가격
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text
    return {'품목':prd_name, '가격':price,'세일가격':sale_price}

In [50]:
# 초기 호출 함수
def get_page_product(url):
    global df_fin
    # 요청 및 파싱 객체 생성
    bs_obj = get_request_product(url) 
    # 페이지내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    
    # 1페이지가 아니면 추천제품은 다시 크롤링 하지 않게 조건 식 생성
    if url.split('=')[1] != '1':
        boxes = boxes[5:len(boxes)]
        
    # 각 제품에 대한 정보 추출
    for box in boxes:
        res = pd.DataFrame(get_product_info(box),index=range(1,2))
        
        df_fin = pd.concat([df_fin,res],axis=0,ignore_index=True)

In [63]:
base_url ='https://jolse.com/category/toners-mists/1019?page='
for i in range(1,5):
    url = base_url + str(i)
    get_page_product(url)

In [64]:
df_fin

,품목,가격,세일가격
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,[+FREE GIFT♥] Haruharu WONDER Black Rice Hyal...,USD 37.00,USD 15.99
...,...,...,...
360,MAMONDE Retinol Ampoule Toner 250ml,USD 35.00,USD 29.75
361,NACIFIC Origin Red Salicylic Acid Toner 150ml,USD 24.00,USD 20.40
362,SKIN1004 Madagascar Centella Poremizing Clear...,USD 21.00,USD 16.80
363,ROUND LAB 1025 Dokdo On The Go Kit,USD 25.00,USD 21.25


In [65]:
df_fin[df_fin['품목']==df_fin['품목'][0]]

,품목,가격,세일가격
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
25,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
50,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
195,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
280,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70


In [44]:
url = 'https://jolse.com/category/toners-mists/1019?page=2' 

In [45]:
url.split('=')[1] !='1'

True

In [55]:
# 마지막 페이지 추출
import ssl
context = ssl._create_unverified_context()
url = 'https://jolse.com/category/toners-mists/1019/'
html = urlopen(url, context=context)
htmls = html.read()


In [59]:
# 2. 파싱객체 생성
bs_obj = BeautifulSoup(htmls,'html.parser')
# >> <a class = last href = https://jolse.com/category/toners-mists/1019?page=27> >> </a>
last_page = bs_obj.find('a',{'class':'last'})
last_page
last_page = bs_obj.find('a',{'class':'last'})['href']
last_page
last_page = bs_obj.find('a',{'class':'last'})['href'].split('=')[1]
last_page
int(last_page)

29

In [66]:
base_url ='https://jolse.com/category/toners-mists/1019?page='
for i in range(1,int(last_page)+1):
    url = base_url + str(i)
    get_page_product(url)

In [ ]:
base_url = 
# 2. 파싱객체 생성
bs_obj = BeautifulSoup(htmls,'html.parser')
# >> <a class=last href = ?page=27> >> </a>
last_page = bs_obj.find('a',{'class':'last'})
last_page
last_page = bs_obj.find('a',{'class':'last'})['href']
last_page
last_page = bs_obj.find('a',{'class':'last'})['href'].split('=')[1]
last_page
int(last_page)

# 마지막 페이지 추출
import ssl
context = ssl._create_unverified_context()
url = 'https://jolse.com/category/toners-mists/1019/'
html = urlopen(url, context=context)
htmls = html.read()
